# CNN model training

## LAQN air quality prediction

## what this notebook does?

this notebook trains a convolutional neural network (CNN) to predict nitrogen dioxide (NO2) levels using the same LAQN data that was used for random forest. the goal is to compare CNN performance against the random forest baseline (test R² = 0.814).

## 1. setup and imports

First, as usual import everything. 
tensorflow/keras is the deep learning library. 
numpy handles arrays. 
matplotlib and seaborn make plots.

In [1]:
# std libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

# scikit-learn for metrics r^2, MSE, MAE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# tensorflow and keras for neural network
import tensorflow as tf
from tensorflow import keras
import platform

print(f'operating system: {platform.system()}')
print(f'processor: {platform.processor()}')
print(f'tensorflow version: {tf.__version__}')
print(f'built with CUDA: {tf.test.is_built_with_cuda()}')

# check all available devices
print('\navailable devices:')
for device in tf.config.list_physical_devices():
    print(f'  - {device}')

operating system: Darwin
processor: arm
tensorflow version: 2.16.2
built with CUDA: False

available devices:
  - PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


        operating system: Darwin
        processor: arm
        tensorflow version: 2.16.2
        built with CUDA: False

        available devices:
        - PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')

In [7]:
# set paths update this to match your folder structure using cwd
base_dir = Path.cwd().parent.parent / 'data' / 'laqn'
data_dir = base_dir / 'ml_prep' # where ml_prep saved the arrays
output_dir = base_dir / 'cnn_model' # where we save CNN outputs 
output_dir.mkdir(parents=True, exist_ok=True)

print(f'loading data from: {data_dir}')
print(f'saving outputs to: {output_dir}')

loading data from: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/ml_prep
saving outputs to: /Users/burdzhuchaglayan/Desktop/data science projects/air-pollution-levels/data/laqn/cnn_model



### GPU availability

- TensorFlow code, and tf.keras models will transparently run on a single GPU with no code changes required.
> Note: Use tf.config.list_physical_devices('GPU') to confirm that TensorFlow is using the GPU.
- The simplest way to run on multiple GPUs, on one or many machines, is using Distribution Strategies.

Source: *Use a GPU :  Tensorflow Core* (no date) *TensorFlow*



In [2]:
# checks gpu availability taken from documentation.

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f'GPU available: {len(gpus)} device(s)')
    for gpu in gpus:
        print(f'  - {gpu.name}')
else:
    print('no GPU found, using CPU (training will be slower but still works)')

no GPU found, using CPU (training will be slower but still works)


## 2. load prepared data

the data was prepared in ml_prep notebook. it created sequences where each sample has 12 hours of history to predict the next hour. this is the same data random forest used, just in 3D shape instead of flattened.

### why 3D data for CNN?

random forest needs flat 2D data: (samples, features). CNN needs 3D data: (samples, timesteps, features). the 3D shape lets CNN learn patterns across time, not just treat each timestep as an independent feature.

think of it like this:
- 2D (random forest): each row is a list of 468 numbers with no structure
- 3D (CNN): each sample is a 12×39 grid where rows are hours and columns are features

In [8]:
# load the 3d sequences for cnn
X_train = np.load(data_dir / 'X_train.npy')
X_val = np.load(data_dir / 'X_val.npy')
X_test = np.load(data_dir / 'X_test.npy')

y_train = np.load(data_dir / 'y_train.npy')
y_val = np.load(data_dir / 'y_val.npy')
y_test = np.load(data_dir / 'y_test.npy')

#load feature_names and scaler
feature_names = joblib.load(data_dir / 'feature_names.joblib')
scaler = joblib.load(data_dir / 'scaler.joblib')

print('data loaded successfully')
print(f'\nshapes:')
print(f'X_train:{X_train.shape}')
print(f'X_val:{X_val.shape}')
print(f'X_test:{X_test.shape}')
print(f'y_train:{y_train.shape}')
print(f'y_val:{y_val.shape}')
print(f'y_test:{y_test.shape}')

data loaded successfully

shapes:
X_train:(9946, 12, 39)
X_val:(2131, 12, 39)
X_test:(2132, 12, 39)
y_train:(9946, 39)
y_val:(2131, 39)
y_test:(2132, 39)


    data loaded successfully

    shapes:
    X_train:(9946, 12, 39)
    X_val:(2131, 12, 39)
    X_test:(2132, 12, 39)
    y_train:(9946, 39)
    y_val:(2131, 39)
    y_test:(2132, 39)

### understanding the shapes

X_train shape is (9946, 12, 39). this means:

| dimension | value | what it represents |
|-----------|-------|-------------------|
| samples | 9,946 | individual training examples |
| timesteps | 12 | hours of history per sample |
| features | 39 | 24 NO2 + 8 PM10 + 3 O3 + 4 temporal |

y_train shape is (9946, 39). the model predicts all 39 features for the next hour.

for fair comparison with random forest, I focus on EN5_NO2 (first column) as the single target. this is the same station used in the RF training report. EN5 had the highest data coverage (99.6%) which makes it the most reliable target for evaluation.

the 3D shape is the key difference from random forest:
- random forest got flattened 2D: (9946, 468) where 468 = 12 × 39
- CNN keeps the 3D structure: (9946, 12, 39)

why does this matter? CNN can learn that hour 1 connects to hour 2 connects to hour 3. random forest just sees 468 separate numbers with no time relationship. this is why CNN might capture temporal patterns better.